In [1]:
from dotenv import load_dotenv,dotenv_values
from datetime import datetime
import requests
import os
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [2]:
load_dotenv()
config = dotenv_values(".env")
os.system("rm *.csv")
api_key = os.getenv("API_KEY_SISTEMA")
url = os.getenv("URL_SISTEMA")


rm: *.csv: No such file or directory


In [3]:
# Headers with the API token
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Adjust the content type as needed
}

# Make a GET request with headers
response = requests.get(url, headers=headers)
print(response)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response data in JSON format
    data_final = response.json()
    print('API Data:', data_final)
else:
    # Print an error message if the request was not successful
    print(f'Error in the request. Status code: {response.status_code}')
    print('Error message:', response.text)

<Response [200]>
API Data: {'success': True, 'data': {'items': [{'id': 1246, 'item_id': 1246, 'name': None, 'full_description': 'ACC-BAND-AW-44-NEG - Accesorios BANDA DE SILICONA WATCH, 44MM, NEGRO', 'description': 'Accesorios BANDA DE SILICONA WATCH, 44MM, NEGRO', 'currency_type_id': 'PEN', 'internal_id': 'ACC-BAND-AW-44-NEG', 'item_code': '', 'currency_type_symbol': 'S/', 'sale_unit_price': '10.00', 'purchase_unit_price': 0, 'unit_type_id': 'NIU', 'sale_affectation_igv_type_id': '10', 'purchase_affectation_igv_type_id': '10', 'calculate_quantity': False, 'has_igv': True, 'is_set': False, 'aux_quantity': 1, 'barcode': 'ACC-BAND-AW-44-NEG', 'factory_code': '', 'brand_id': None, 'brand': '', 'category_id': 1, 'category': 'Accesorios', 'stock': 0, 'image': 'https://controlerp.catapu.com/logo/imagen-no-disponible.jpg', 'warehouses': [{'warehouse_description': 'Almacén Oficina Principal', 'stock': '0.0000', 'warehouse_id': 1}, {'warehouse_description': 'Almacén - Online', 'stock': '0.0000'

json_data = data_final

In [4]:
rows = []
for items in data_final['data']['items']:


    if items['category'] == 'iPhone':
        ri = items['internal_id']
        split = ri.split("-")
        if len(split) >= 2:
            modelo = f'{split[0]}-{split[1]}-{split[2]}'
        else:
            modelo = 'revisar'
        precio = float(items['sale_unit_price'])

        nuevo_precio = precio + 40

        online = float(items['warehouses'][1]['stock'])
        polo = float(items['warehouses'][2]['stock'])
        total = online + polo
        rows.append([modelo,ri,online,polo,total])

In [5]:
df = pd.DataFrame(rows, columns=["modelo", "sku", "tienda_online", "tienda_polo","total"])
df = df.drop(columns='total')
df = pd.melt(df, id_vars=['modelo','sku'], value_vars=['tienda_online','tienda_polo'],var_name='ubicacion',value_name='inventario')
df[df['inventario'] < 0]
df = df[df['inventario'] > 0]


In [6]:
# df['Grado'] = df['sku'].str.split("-").str[3]
# df['Color'] = df['sku'].str.split("-").str[4]
# df


In [7]:

# def change_grado(x):
#     grado = {'A+':'1.A+', 'A':'2.A', 'B':'3.B', 'C':'4.C'}
#     if x in grado:
#         return grado[x]
#     else:
#         return 'No existe grado'


In [8]:
# df['Grado'] = df['Grado'].apply(change_grado)
# df

In [9]:
# new_df =  pd.pivot_table(df,values='inventario', index=['modelo','Color'],columns=['ubicacion','Grado'], aggfunc='sum').reset_index()
# new_df

In [17]:
fecha_actual = datetime.now()
nombre_fecha = fecha_actual.strftime("%d%m%y %H:%m")
csv_name = f'{nombre_fecha}-stock.csv'

100524 13:05


In [11]:
df.to_csv(csv_name, sep=',', index=False, encoding='utf-8-sig')

In [ ]:
# Configuración del servidor SMTP
api_key = config["API_KEY_SISTEMA"]

smtp_server = 'smtpout.secureserver.net'
smtp_port = 587  # Puerto SMTP (generalmente 587 o 465)
smtp_username = config["EMAIL_USER"]
smtp_password = config["EMAIL_PASS"]

# Configuración del mensaje
sender = config["EMAIL_USER"]
receiver = ['email@ss.com','emai2@ss.com']
subject = f'Ventas del {intial_date}-{final_date} Facturador'
body = 'Se envia los Documentos emitidos en los puntos de Venta'
archivo_adjunto = zip_file_path

# Crear el mensaje
message = MIMEMultipart()
message['From'] = sender
message['To'] = ','.join(receiver)
message['Subject'] = subject
message.attach(MIMEText(body, 'plain'))

# Adjuntar archivo ZIP
with open(archivo_adjunto, 'rb') as f:
    part = MIMEApplication(f.read(), Name=archivo_adjunto)
    part['Content-Disposition'] = f'attachment; filename="{archivo_adjunto}"'
    message.attach(part)

# Iniciar sesión en el servidor SMTP y enviar el correo
with smtplib.SMTP(smtp_server, smtp_port, timeout=5) as server:
    server.starttls()  # Habilitar cifrado TLS
    server.login(smtp_username, smtp_password)
    server.sendmail(sender, receiver, message.as_string())

print('Correo electrónico enviado con éxito.')